# Skin Cancer

# Imports

In [1]:
import torch
import numpy as np
from torch.autograd import backward
import torch.nn as nn
from torch.nn.modules import ReLU
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

## Dataset

In [2]:
#Pasando de Imagen a Tensor
transform = transforms.Compose([
transforms.Resize((224,224)),
transforms.ToTensor()
])

#Definiendo que es el Dataset y preparando el DataLoader
directory = './DermMel'
train_dataset = datasets.ImageFolder(root="./DermMel/train_sep",transform=transform)
train_loader = DataLoader(train_dataset,batch_size=64,shuffle=True)
test_dataset = datasets.ImageFolder(root="./DermMel/test",transform=transform)
test_loader = DataLoader(test_dataset,batch_size=64,shuffle=True)

## Modelo

In [3]:
#Ejemplo del train dataset print(train_dataset[10][0][RGB,px,py])
#Modelo
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f'Device: {device}')
class MelanomaDetector(nn.Module):
    def __init__ (self):
        super().__init__ ()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=32,kernel_size=5,stride=2),
            nn.MaxPool2d(kernel_size=2,stride=2,padding=0),
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=2),
            nn.MaxPool2d(kernel_size=2,stride=2,padding=0),
            nn.Flatten(),
            nn.Linear(10816,1000),
            nn.ReLU(),
            nn.Linear(1000,100),
            nn.ReLU(),
            nn.Linear(100,2)
        )
    def forward(self,X):
        X = self.features(X)
        return X

#Instancia del modelo 
model = MelanomaDetector().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)

Device: cuda


## Train

In [4]:
#Funcion de entrenamiento
def train_model(model,train_loader,criterion,optimizer,num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs,labels = inputs.to(device),labels.to(device)

            #Paso forward
            outputs = model(inputs)
            loss = criterion(outputs,labels)

            #Paso backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')
train_model(model,train_loader,criterion,optimizer)

#Funcion de eval
def evaluate_model(model,test_loader):
    model.eval()
    correct = 0
    correct_sum = 0
    total = 0
    total_sum = 0
    for inputs,labels in test_loader:
            inputs,labels = inputs.to(device),labels.to(device)
            outputs = model(inputs)
            _,predicted = torch.max(outputs.data,1)
            total =+ labels.size(0)
            total_sum =+ total
            correct =+ (predicted == labels).sum().item()
            correct_sum =+ correct
            print(f'Accuracy of the network on the {total} test images: {100 * correct / total:.2f}%')

    print(f'TotalTotal  Accuracy of the network on the {total_sum} test images: {100 * correct_sum / total_sum:.2f}%')
evaluate_model(model,test_loader)
torch.save(model,'skincancer.pth')

Epoch [1/5], Loss: 0.0041
Epoch [1/5], Loss: 0.0179
Epoch [1/5], Loss: 0.0221
Epoch [1/5], Loss: 0.0262
Epoch [1/5], Loss: 0.0302
Epoch [1/5], Loss: 0.0340
Epoch [1/5], Loss: 0.0375
Epoch [1/5], Loss: 0.0414
Epoch [1/5], Loss: 0.0497
Epoch [1/5], Loss: 0.0536
Epoch [1/5], Loss: 0.0590
Epoch [1/5], Loss: 0.0634
Epoch [1/5], Loss: 0.0680
Epoch [1/5], Loss: 0.0725
Epoch [1/5], Loss: 0.0765
Epoch [1/5], Loss: 0.0806
Epoch [1/5], Loss: 0.0847
Epoch [1/5], Loss: 0.0886
Epoch [1/5], Loss: 0.0927
Epoch [1/5], Loss: 0.0966
Epoch [1/5], Loss: 0.1005
Epoch [1/5], Loss: 0.1042
Epoch [1/5], Loss: 0.1077
Epoch [1/5], Loss: 0.1114
Epoch [1/5], Loss: 0.1153
Epoch [1/5], Loss: 0.1191
Epoch [1/5], Loss: 0.1231
Epoch [1/5], Loss: 0.1274
Epoch [1/5], Loss: 0.1315
Epoch [1/5], Loss: 0.1355
Epoch [1/5], Loss: 0.1391
Epoch [1/5], Loss: 0.1432
Epoch [1/5], Loss: 0.1471
Epoch [1/5], Loss: 0.1511
Epoch [1/5], Loss: 0.1548
Epoch [1/5], Loss: 0.1586
Epoch [1/5], Loss: 0.1628
Epoch [1/5], Loss: 0.1665
Epoch [1/5],

# Extra

In [5]:
def saveTrainedModel():
    torch.save(model,'skincancer.pth')

def chargeTrainedModel():
    model = MelanomaDetector()
    model.load_state_dict(torch.load('skincancer.pth'))
    #model.eval()
    return model


model = torch.load("skincancer.pth")
#print(model)

saveTrainedModel()

/tmp/ipykernel_5534/693354255.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("skincancer.pth")


## Individual

In [6]:
from PIL import Image
from torchvision import transforms, io
def individual(path):
    model = chargeTrainedModel().to(device)
    image_input = io.read_image(path).to(device)

    image_input = image_input.reshape(1,3,224,224)/255
    
    print(image_input.shape)
    logits = model(image_input) # [1, 3, 224, 224]
    
    print(logits)
    probabilities = torch.softmax(logits, dim=1)
    print(probabilities)
    if torch.argmax(probabilities) == 0:
        result = "cancer"
    else:
        result = "no cancer"
    print("Enhorabona, el resultat és: ", result)






In [7]:
individual("./prueba/AUG_0_1.jpeg")

/tmp/ipykernel_5534/693354255.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('skincancer.pth'))


TypeError: Expected state_dict to be dict-like, got <class '__main__.MelanomaDetector'>.

In [ ]:
tensor = torch.tensor([[10,0,0,0]], dtype=torch.float32)
print(torch.softmax(tensor, dim=1))